## MH4518 Group Project

Product: https://derivative.credit-suisse.com/ch/ch/en/detail/drop-back-certificate-s-p-500/CH1199361879/119936187


### 1. Initial Set-up

<ol>
<li> Set directories
<li> Install dependencies
<li> Import libraries
</ol>

NOTE: Ensure you are using the correct kernel to run the Jupyter Notebook with the correctly set root driectory.

In [62]:
import os, sys

# Set the root directory accordingly
ROOT_DIR = "d:/Coding/MH4518"
os.chdir(ROOT_DIR)
sys.path.append(ROOT_DIR)

# The data directory is set relative to root
DATA_DIR = os.path.join(ROOT_DIR, "./data")

In [64]:
# Install additional libraries using pip
#! pip install -r requirements.txt

# Or if you have Conda installed (reccommended)
# conda env create -f environment.yml

Note: you may need to restart the kernel to use updated packages.


& was unexpected at this time.


In [65]:
import numpy as np
import pandas as pd

from numba import jit

In [66]:
# OPTIONAL: Suppresing the Deprecation Notices while using numba (reccommended)
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

### 2. Loading Data

The asset and product price data was collected and preprocessed using the scripts present in the `/data` directory.
<li> Start date: 09-08-2022
<li> End date: 09-11-2023
<br>

TODO: Update datasets (currently only till Oct 20)

In [73]:
asset_prices = pd.read_csv(os.path.join(DATA_DIR, "asset_price_1_year.csv"))
asset_prices["date"] = pd.to_datetime(asset_prices["date"])
asset_prices = asset_prices.set_index("date")

product_prices = pd.read_csv(os.path.join(DATA_DIR, "product_price_1_year.csv"))
product_prices["date"] = pd.to_datetime(product_prices["date"])
product_prices = product_prices.set_index("date")

# Sanity check
assert len(asset_prices) == len(product_prices)

### 3. Payoff Function

In [44]:
def find_trigger_price(S_path, trigger):
    '''
    Returns the price that triggered the event, if it exists, else None.
    '''
    # Return the first price that triggers the event
    for price in S_path:
        if price < trigger:
            return price
        
    return None


def calculate_payoff(S_path, Y_0 = 1000, rate = 0.0985, years = 3):
    '''
    Computes the payoff of a simulated path.
    '''
    S_0 = S_path[0]
    S_T = S_path[-1]
    S_t1 = find_trigger_price(S_path, trigger = 0.90*S_0)
    S_t2 = find_trigger_price(S_path, trigger = 0.85*S_0)
    S_t3 = find_trigger_price(S_path, trigger = 0.80*S_0)

    # 1. Fixed returns independent of trigger events
    Y_T = (0.55*Y_0) * (S_T/S_0)    

    # 2. Adjusted returns after trigger events
    if S_t3 is not None:
        multiplier = (S_T/S_t1 + S_T/S_t2 + S_T/S_t3)
    elif S_t2 is not None:
        multiplier = (S_T/S_t1 + S_T/S_t2)
    elif S_t1 is not None:
        multiplier = (S_T/S_t1)
    else:
        multiplier = 0.0
    Y_T += (0.15*Y_0) * multiplier

    # 3. Daily accrued interest of 9.85% p.a.
    if S_t3 is not None:
        principal = 0.0
    elif S_t2 is not None:
        principal = (0.15*Y_0)
    elif S_t1 is not None:
        principal = (0.30*Y_0)
    else:
        principal = (0.45*Y_0)
    Y_T += principal * rate * years
    
    return Y_T

In [45]:
# OPTIONAL: Testing the payoff function with values in the factsheet
S_0 = 3790.38
S_path_partial = [3790.41, 3800.15, 3373.44, 4000.00, 3183.92, 2994.40]
S_T = [0.0, 2653.27, 3032.3, 3411.34, 3790.38, 4169.42, 4548.46, 4927.49]

for final_closing_price in S_T:
    S_path = S_path_partial + [final_closing_price]
    payoff = calculate_payoff(S_path)

    print(f"S_T = {final_closing_price} \t   Performance of Asset = {100*(final_closing_price - S_0)/S_0:.2f}% \t Performance of Certificate = {(payoff - 1000)/10:.2f}% \t Payoff = {payoff:.2f}")

S_T = 0.0 	   Performance of Asset = -100.00% 	 Performance of Certificate = -100.00% 	 Payoff = 0.00
S_T = 2653.27 	   Performance of Asset = -30.00% 	 Performance of Certificate = -23.91% 	 Payoff = 760.89
S_T = 3032.3 	   Performance of Asset = -20.00% 	 Performance of Certificate = -13.04% 	 Payoff = 869.58
S_T = 3411.34 	   Performance of Asset = -10.00% 	 Performance of Certificate = -2.17% 	 Payoff = 978.28
S_T = 3790.38 	   Performance of Asset = 0.00% 	 Performance of Certificate = 8.70% 	 Payoff = 1086.98
S_T = 4169.42 	   Performance of Asset = 10.00% 	 Performance of Certificate = 19.57% 	 Payoff = 1195.68
S_T = 4548.46 	   Performance of Asset = 20.00% 	 Performance of Certificate = 30.44% 	 Payoff = 1304.38
S_T = 4927.49 	   Performance of Asset = 30.00% 	 Performance of Certificate = 41.31% 	 Payoff = 1413.07


### 4. Geometric Brownian Motion

Implmentation of the Black-Scholes model for pricing. THe form derived from Ito's lemma was used.

In [67]:
@jit  # TODO: Suppress numba warnings.
def simulate_GBM_exact(num_sim, S_0, r, sigma, delta_t, T, Z=None):
  # Initializations
  v = r - 0.5*(sigma**2)
  num_periods = int(T/delta_t)
  S_matrix = np.zeros(shape=(num_sim, num_periods + 1))

  # Perform the sampling from the standard normal dsitribution
  if(type(Z) != np.ndarray):
    Z = np.random.normal(0, 1, size=(num_sim, num_periods))

  # We are using the form derived by Ito's lemma 
  for i in range(num_sim):
    S_matrix[i][0] = S_0
    for j in range(1, num_periods + 1):
      #log_diff = mu*delta_t - 0.5*(sigma**2)*delta_t + (sigma*np.sqrt(delta_t) * Z[i][j-1])
      log_diff = v*delta_t + (sigma*np.sqrt(delta_t) * Z[i][j-1])
      S_matrix[i][j] = S_matrix[i][j-1]*np.exp(log_diff)

  return S_matrix

### 5. Variance Reduction for GBM

Functions that implement,

<ol> 
<li> Antithetic Variates
<li> Control Variates
<li> Empirical Martingale Simulation
</ol>

In [68]:
@jit
def simulate_GBM_exact_AV(num_sim, S_0, r, sigma, delta_t, T, Z=None):
  # Initializations
  v = r - 0.5*(sigma**2)
  num_periods = int(T/delta_t)
  S_matrix = np.zeroes(shape=(num_sim, num_periods + 1))
  S_tilde_matrix = np.zeroes(shape=(num_sim, num_periods + 1))

  # Perform the sampling from the standard normal dsitribution
  if(type(Z) != np.ndarray):
    Z = np.random.normal(0, 1, size=(num_sim, num_periods))

  # Ito's lemma form (just a bit more concise)
  for i in range(num_sim):
    S_matrix[i][0] = S_0
    S_tilde_matrix[i][0] = S_0
    for j in range(1, num_periods + 1):
        S_matrix[i][j] = S_matrix[i][j-1]*np.exp(v*delta_t + (sigma*np.sqrt(delta_t) * Z[i][j-1]))
        S_tilde_matrix[i][j] = S_tilde_matrix[i][j-1]*np.exp(v*delta_t - (sigma*np.sqrt(delta_t) * Z[i][j-1]))  # Negative Z(.)  
  
  return np.vstack((S_matrix, S_tilde_matrix))

In [69]:
# TODO: Control Variates

In [70]:
@jit
def simulate_GBM_exact_EMS(num_sim, S_0, r, sigma, delta_t, T, Z=None):
  # Initializations
  num_periods = int(T/delta_t)
  S_matrix = np.zeroes(shape=(num_sim, num_periods + 1))
  Z_matrix = np.zeroes(shape=(num_sim + 1, num_periods + 1))

  # Simulating the GBM path and set the first coloumn of S_matrix
  if(type(Z) == np.ndarray):
    gbm_path = simulate_GBM_exact(num_sim, S_0, r, sigma, delta_t, T, Z=Z)
  else:
    gbm_path = simulate_GBM_exact(num_sim, S_0, r, sigma, delta_t, T)
    
  S_matrix[:, 0] = gbm_path[:, 0]

  # Correcting the path using EMS
  for j in range(1, num_periods+1):
    Z_matrix[:num_sim, j-1] = (S_matrix[:, j-1] * gbm_path[:, j])/gbm_path[:, j-1]
    Z_matrix[num_sim, j-1] = np.exp(-r * ((j-1) * delta_t)) * np.mean(Z_matrix[:num_sim, j-1])
    S_matrix[:, j] = (gbm_path[:, 0] * Z_matrix[:num_sim, j-1])/Z[num_sim, j-1]

  return S_matrix

### 6. Backtesting GBM

We take 252 day long windows of data for each iteration.

eg: The first window begins on 9th August, 2022 and ends on 9th August, 2023 and we price the product on 10th August.

In [85]:
# Parameters for each set of simulations
#last_window_start_date = pd.to_datetime("2022-11-09")  # TODO: Update dataset
last_window_start_date = pd.to_datetime("2022-10-20")

num_sim = 10000
delta_t = 1/252
T = 1

In [107]:
for window_start_date in asset_prices.index:
    # Termination condiditon
    if window_start_date > last_window_start_date:
        break
    
    # CHECK: Some of the windows have 251 data points 
    window_end_date = window_start_date + pd.DateOffset(years=1)
    window_asset_data = asset_prices[(asset_prices.index >= window_start_date) & (asset_prices.index < window_end_date)]  # Strict inequality for window_end_date

    
